In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("MTA ACE Violations - Kaggle Analysis")
print("Dataset: sneharoy05/mta-bus-ace-violations-cuny-datathon-2025")
print("=" * 70)

MTA ACE Violations - Kaggle Analysis
Dataset: sneharoy05/mta-bus-ace-violations-cuny-datathon-2025


In [2]:
file_path = "MTA_Bus_Automated_Camera_Enforcement_Violations__Beginning_October_2019_20250922.csv"

try:
    df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "sneharoy05/mta-bus-ace-violations-cuny-datathon-2025",
        file_path
    )
    print(f"✅ Successfully loaded {len(df):,} rows and {len(df.columns)} columns")
    print("First 5 records:")
    print(df.head())
    
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("Falling back to sample data for demonstration...")

✅ Successfully loaded 1,667,477 rows and 15 columns
First 5 records:
   Violation ID                                         Vehicle ID  \
0     489749182  c5ae1411153b52556a1e648cc80d718aa519a4bdd189ab...   
1     489744714  df9044acf85cf55488aea4cd3ce1d0e17ef050551726b6...   
2     489743631  eb5a337966ba65f66ab1db8e169d2446a4fb429b0efc63...   
3     489741945  3f877f70d9b253515a945be807c9c62d5814949f810310...   
4     489741940  7feac037b62d591ffb1214e356157f3dd197fc22fee5bb...   

         First Occurrence         Last Occurrence            Violation Status  \
0  08/20/2025 11:12:08 PM  08/21/2025 12:24:08 AM       TECHNICAL ISSUE/OTHER   
1  08/20/2025 11:48:59 PM  08/20/2025 11:54:47 PM    EXEMPT - BUS/PARATRANSIT   
2  08/20/2025 10:33:13 PM  08/20/2025 11:56:02 PM       TECHNICAL ISSUE/OTHER   
3  08/20/2025 10:50:45 PM  08/20/2025 11:32:43 PM              EXEMPT - OTHER   
4  08/20/2025 10:52:57 AM  08/20/2025 11:16:57 AM  EXEMPT - EMERGENCY VEHICLE   

         Violation Type

In [3]:
# Clean column names and data
df.columns = df.columns.str.strip()
if 'First Occurrence' in df.columns:
    df['First Occurrence'] = pd.to_datetime(df['First Occurrence'])

print(f"\n Dataset columns: {list(df.columns)}")


 Dataset columns: ['Violation ID', 'Vehicle ID', 'First Occurrence', 'Last Occurrence', 'Violation Status', 'Violation Type', 'Bus Route ID', 'Violation Latitude', 'Violation Longitude', 'Stop ID', 'Stop Name', 'Bus Stop Latitude', 'Bus Stop Longitude', 'Violation Georeference', 'Bus Stop Georeference']


In [4]:
#Mapping
if all(col in df.columns for col in ['Violation Latitude', 'Violation Longitude']):
    valid_coords = df[
        (df['Violation Latitude'].notna()) & 
        (df['Violation Longitude'].notna()) &
        (df['Violation Latitude'] != 0) & 
        (df['Violation Longitude'] != 0) &
        (df['Violation Latitude'].between(40.4, 41.0)) &  # NYC bounds
        (df['Violation Longitude'].between(-74.5, -73.5))
    ]
    print(f"📍 {len(valid_coords):,} violations with valid NYC coordinates")
else:
    print("❌ Coordinate columns not found, using all data")
    valid_coords = df

📍 1,667,477 violations with valid NYC coordinates


In [5]:
def create_cuny_routes_map():
    print("\n Business Question 1: CUNY Routes Analysis")
    print("-" * 50)
    
    # CUNY campus locations and their primary serving routes
    cuny_campuses = {
        'Hunter College': {
            'lat': 40.7685, 'lon': -73.9656, 
            'routes': ['M101', 'M103', 'M98', 'M102'],
            'students': 23000
        },
        'Brooklyn College': {
            'lat': 40.6313, 'lon': -73.9537,
            'routes': ['B12', 'B49', 'B16'],
            'students': 18000
        },
        'Queens College': {
            'lat': 40.7362, 'lon': -73.8194,
            'routes': ['Q17', 'Q20A', 'Q25', 'Q34'],
            'students': 20000
        },
        'Baruch College': {
            'lat': 40.7402, 'lon': -73.9834,
            'routes': ['M23', 'M15', 'M101', 'M102'],
            'students': 19000
        },
        'City College': {
            'lat': 40.8200, 'lon': -73.9495,
            'routes': ['M3', 'M11', 'M18', 'Bx19'],
            'students': 16000
        },
        'John Jay College': {
            'lat': 40.7700, 'lon': -73.9886,
            'routes': ['M11', 'M104', 'M7'],
            'students': 15000
        }
    }
    
    # Map centered on NYC
    m = folium.Map(
        location=[40.7128, -73.9853], 
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    title_html = '''
                 <h3 align="center" style="font-size:20px"><b>CUNY Campus Bus Route Violations</b></h3>
                 <p align="center">Interactive map showing violations on routes serving CUNY students</p>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # All CUNY routes from our data
    all_cuny_routes = []
    for campus_info in cuny_campuses.values():
        all_cuny_routes.extend(campus_info['routes'])
    cuny_routes_set = set(all_cuny_routes)
    
    # Violations for CUNY routes
    if 'Bus Route ID' in valid_coords.columns:
        cuny_violations = valid_coords[valid_coords['Bus Route ID'].isin(cuny_routes_set)]
        print(f"📊 Found {len(cuny_violations):,} violations on CUNY-serving routes")
        
        # Route-specific 
        route_stats = cuny_violations['Bus Route ID'].value_counts()
        print(f"Top CUNY routes by violations: {dict(route_stats.head())}")
    else:
        cuny_violations = valid_coords.head(1000)  # Sample for demo
    
    # CUNY campuses with detailed popups
    for campus, info in cuny_campuses.items():
        # Calculate violations near this campus (within ~1km)
        if len(cuny_violations) > 0 and all(col in cuny_violations.columns for col in ['Violation Latitude', 'Violation Longitude']):
            nearby_violations = cuny_violations[
                (abs(cuny_violations['Violation Latitude'] - info['lat']) < 0.01) &
                (abs(cuny_violations['Violation Longitude'] - info['lon']) < 0.01)
            ]
            nearby_count = len(nearby_violations)
        else:
            nearby_count = 0
        
        # Detailed popup
        popup_html = f"""
        <div style="width:250px">
            <h4><b>{campus}</b></h4>
            <p><b>Student Population:</b> {info['students']:,}</p>
            <p><b>Primary Bus Routes:</b> {', '.join(info['routes'])}</p>
            <p><b>Nearby Violations:</b> {nearby_count:,}</p>
            <p><b>Location:</b> {info['lat']:.4f}, {info['lon']:.4f}</p>
        </div>
        """
        
        folium.Marker(
            [info['lat'], info['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='red', icon='graduation-cap', prefix='fa'),
            tooltip=f"{campus} - {info['students']:,} students"
        ).add_to(m)
        
        # Campus influence circle
        folium.Circle(
            [info['lat'], info['lon']],
            radius=1000,  # 1km radius
            popup=f"{campus} - 1km radius",
            color='red',
            weight=2,
            fillOpacity=0.1
        ).add_to(m)
    
    # Violation heatmap for CUNY routes
    if len(cuny_violations) > 0 and all(col in cuny_violations.columns for col in ['Violation Latitude', 'Violation Longitude']):
        cuny_coords = [
            [row['Violation Latitude'], row['Violation Longitude']] 
            for _, row in cuny_violations.iterrows()
        ]
        
        HeatMap(
            cuny_coords,
            radius=15,
            blur=20,
            max_zoom=18,
            gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'},
            name='CUNY Route Violations Heatmap'
        ).add_to(m)
    
    # M101 specific 
    if 'Bus Route ID' in valid_coords.columns:
        m101_violations = valid_coords[valid_coords['Bus Route ID'] == 'M101']
        if len(m101_violations) > 0:
            m101_cluster = MarkerCluster(name='M101 Route (Top CUNY Route)').add_to(m)
            
            # Sample M101 violations for performance
            sample_m101 = m101_violations.sample(min(200, len(m101_violations)))
            
            for _, row in sample_m101.iterrows():
                popup_text = f"""
                <b>M101 Route Violation</b><br>
                Date: {row.get('First Occurrence', 'N/A')}<br>
                Status: {row.get('Violation Status', 'N/A')}<br>
                Type: {row.get('Violation Type', 'N/A')}
                """
                
                folium.CircleMarker(
                    [row['Violation Latitude'], row['Violation Longitude']],
                    popup=popup_text,
                    radius=4,
                    color='blue',
                    fillColor='lightblue',
                    fillOpacity=0.7
                ).add_to(m101_cluster)
    
    
    folium.LayerControl().add_to(m)
    
    return m

In [6]:
# Question 2: Exempt vehicle Hotspot MAP
def create_exempt_violations_map():
    print("\n🚫 Business Question 2: Exempt Vehicle Violations")
    print("-" * 50)
    
    # Filter exempt violations
    if 'Violation Status' in valid_coords.columns:
        exempt_violations = valid_coords[
            valid_coords['Violation Status'].str.contains('EXEMPT', na=False, case=False)
        ]
        print(f"🚫 Found {len(exempt_violations):,} exempt violations")
    else:
        exempt_violations = valid_coords.sample(min(5000, len(valid_coords)))
        print("📊 Using sample data for exempt analysis")
    
    # Create base map
    m = folium.Map(
        location=[40.7128, -73.9853], 
        zoom_start=11,
        tiles='CartoDB dark_matter'
    )
    
    # Add title
    title_html = '''
                 <h3 align="center" style="font-size:20px; color:white;"><b>Exempt Vehicle Violation Hotspots</b></h3>
                 <p align="center" style="color:white;">Geographic patterns of repeat exempt offenders</p>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Create violation heatmap
    if len(exempt_violations) > 0 and all(col in exempt_violations.columns for col in ['Violation Latitude', 'Violation Longitude']):
        exempt_coords = [
            [row['Violation Latitude'], row['Violation Longitude']] 
            for _, row in exempt_violations.iterrows()
        ]
        
        HeatMap(
            exempt_coords,
            radius=12,
            blur=15,
            max_zoom=18,
            gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 0.8: 'red', 1: 'darkred'},
            name='Exempt Violations Heatmap'
        ).add_to(m)
    
    # Analyze repeat offenders
    if 'Vehicle ID' in exempt_violations.columns:
        repeat_offenders = exempt_violations['Vehicle ID'].value_counts()
        top_offenders = repeat_offenders.head(10)
        
        print(f"🔍 Top repeat offenders: {dict(top_offenders.head(5))}")
        
        # Add repeat offender locations
        colors = ['darkred', 'red', 'orange', 'yellow', 'pink']
        
        for i, (vehicle_id, violation_count) in enumerate(top_offenders.items()):
            vehicle_violations = exempt_violations[exempt_violations['Vehicle ID'] == vehicle_id]
            
            if len(vehicle_violations) > 0:
                color = colors[min(i, len(colors)-1)]
                
                # Create cluster for this vehicle
                vehicle_cluster = MarkerCluster(
                    name=f'Vehicle {vehicle_id[:3]}*** ({violation_count} violations)'
                ).add_to(m)
                
                for _, row in vehicle_violations.iterrows():
                    popup_html = f"""
                    <div style="width:200px">
                        <h4><b>Repeat Offender #{i+1}</b></h4>
                        <p><b>Vehicle:</b> {vehicle_id[:3]}***{vehicle_id[-3:]}</p>
                        <p><b>Total Violations:</b> {violation_count}</p>
                        <p><b>Date:</b> {row.get('First Occurrence', 'N/A')}</p>
                        <p><b>Status:</b> {row.get('Violation Status', 'N/A')}</p>
                    </div>
                    """
                    
                    folium.CircleMarker(
                        [row['Violation Latitude'], row['Violation Longitude']],
                        popup=folium.Popup(popup_html, max_width=250),
                        radius=5,
                        color=color,
                        fillColor=color,
                        fillOpacity=0.8
                    ).add_to(vehicle_cluster)
    
    # Add NYC borough boundaries for context
    borough_centers = {
        'Manhattan': [40.7831, -73.9712],
        'Brooklyn': [40.6782, -73.9442],
        'Queens': [40.7282, -73.7949],
        'Bronx': [40.8448, -73.8648],
        'Staten Island': [40.5795, -74.1502]
    }
    
    for borough, coords in borough_centers.items():
        folium.Marker(
            coords,
            popup=f"{borough}<br>Borough Center",
            icon=folium.Icon(color='green', icon='info-sign'),
            tooltip=borough
        ).add_to(m)
    
    folium.LayerControl().add_to(m)
    return m

In [7]:
# Question 3: Congestion Pricing Impact Map
def create_congestion_pricing_map():
    print("\n📈 Business Question  3: Congestion Pricing Impact")
    print("-" * 50)
    
    # Manhattan CBD bounds (congestion pricing zone)
    cbd_bounds = {
        'north': 40.7614,  # 60th Street
        'south': 40.7047,  # Battery Park
        'west': -74.0200,  # Hudson River
        'east': -73.9442   # East River
    }
    
    # CBD violations Filtered
    if all(col in valid_coords.columns for col in ['Violation Latitude', 'Violation Longitude']):
        cbd_violations = valid_coords[
            (valid_coords['Violation Latitude'] >= cbd_bounds['south']) &
            (valid_coords['Violation Latitude'] <= cbd_bounds['north']) &
            (valid_coords['Violation Longitude'] >= cbd_bounds['west']) &
            (valid_coords['Violation Longitude'] <= cbd_bounds['east'])
        ]
        print(f"🏙️ Found {len(cbd_violations):,} violations in CBD")
    else:
        cbd_violations = valid_coords.sample(min(2000, len(valid_coords)))
    
    # Split by congestion pricing 
    congestion_start = pd.to_datetime('2025-01-05')
    
    if 'First Occurrence' in cbd_violations.columns:
        before_cp = cbd_violations[cbd_violations['First Occurrence'] < congestion_start]
        after_cp = cbd_violations[cbd_violations['First Occurrence'] >= congestion_start]
        
        print(f"📊 Before CP (Jan 1-4): {len(before_cp):,} violations")
        print(f"📊 After CP (Jan 5+): {len(after_cp):,} violations")
        
        if len(before_cp) > 0 and len(after_cp) > 0:
            reduction = (1 - len(after_cp)/len(before_cp)) * 100
            print(f"📉 Reduction: {reduction:.1f}%")
    else:
        before_cp = cbd_violations.sample(min(1000, len(cbd_violations)))
        after_cp = cbd_violations.drop(before_cp.index)
    
    m = folium.Map(
        location=[40.7282, -73.9942], 
        zoom_start=13,
        tiles='OpenStreetMap'
    )
    
    title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Congestion Pricing Impact Analysis</b></h3>
                 <p align="center">Before vs After January 5, 2025 implementation</p>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add CBD boundary rectangle
    folium.Rectangle(
        bounds=[
            [cbd_bounds['south'], cbd_bounds['west']], 
            [cbd_bounds['north'], cbd_bounds['east']]
        ],
        popup="""
        <b>Manhattan Central Business District</b><br>
        Congestion Pricing Zone<br>
        Implemented: January 5, 2025<br>
        Boundary: Below 60th Street
        """,
        color='purple',
        weight=3,
        fillOpacity=0.1
    ).add_to(m)
    
    # Before congestion pricing heatmap
    if len(before_cp) > 0 and all(col in before_cp.columns for col in ['Violation Latitude', 'Violation Longitude']):
        before_coords = [
            [row['Violation Latitude'], row['Violation Longitude']] 
            for _, row in before_cp.iterrows()
        ]
        
        HeatMap(
            before_coords,
            radius=8,
            blur=10,
            gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'},
            name=f'Before Congestion Pricing ({len(before_cp):,} violations)'
        ).add_to(m)
    
    # After congestion pricing heatmap
    if len(after_cp) > 0 and all(col in after_cp.columns for col in ['Violation Latitude', 'Violation Longitude']):
        after_coords = [
            [row['Violation Latitude'], row['Violation Longitude']] 
            for _, row in after_cp.iterrows()
        ]
        
        HeatMap(
            after_coords,
            radius=8,
            blur=10,
            gradient={0.2: 'green', 0.4: 'lightgreen', 0.6: 'orange', 0.8: 'red', 1: 'darkred'},
            name=f'After Congestion Pricing ({len(after_cp):,} violations)'
        ).add_to(m)
    
    # Key CBD landmarks
    cbd_landmarks = {
        'Times Square': [40.7580, -73.9855, 'Major Transit Hub'],
        'Wall Street': [40.7074, -74.0113, 'Financial District'],
        'Union Square': [40.7359, -73.9911, 'Shopping & Transit'],
        'Columbus Circle': [40.7681, -73.9819, 'Upper West Side Gateway'],
        'Brooklyn Bridge': [40.7061, -73.9969, 'Major River Crossing'],
        'Lincoln Tunnel': [40.7614, -74.0055, 'NJ Connection']
    }
    
    for landmark, (lat, lon, description) in cbd_landmarks.items():
        folium.Marker(
            [lat, lon],
            popup=f"<b>{landmark}</b><br>{description}",
            icon=folium.Icon(color='blue', icon='info-sign'),
            tooltip=landmark
        ).add_to(m)
    
    folium.LayerControl().add_to(m)
    return m

In [8]:
# All maps
print("\n🗺️ Generating Maps...")

maps_created = []

try:
    cuny_map = create_cuny_routes_map()
    cuny_map.save('cuny_routes_analysis.html')
    maps_created.append('cuny_routes_analysis.html')
    print("✅ CUNY routes map generated")
except Exception as e:
    print(f"❌ Error creating CUNY map: {e}")

try:
    exempt_map = create_exempt_violations_map()
    exempt_map.save('exempt_violations_hotspots.html')
    maps_created.append('exempt_violations_hotspots.html')
    print("✅ Exempt violations map generated!")
except Exception as e:
    print(f"❌ Error creating exempt map: {e}")

try:
    cp_map = create_congestion_pricing_map()
    cp_map.save('congestion_pricing_impact.html')
    maps_created.append('congestion_pricing_impact.html')
    print("✅ Congestion pricing map generated")
except Exception as e:
    print(f"❌ Error creating congestion pricing map: {e}")

# Summarization
print("\n📊 Creating Summary Statistics...")

# Summary insights
summary_stats = {
    'total_violations': len(df),
    'valid_coordinates': len(valid_coords),
    'date_range': f"{df['First Occurrence'].min()} to {df['First Occurrence'].max()}" if 'First Occurrence' in df.columns else "Unknown",
    'unique_routes': df['Bus Route ID'].nunique() if 'Bus Route ID' in df.columns else 0,
    'exempt_violations': len(df[df['Violation Status'].str.contains('EXEMPT', na=False, case=False)]) if 'Violation Status' in df.columns else 0,
    'maps_created': len(maps_created)
}

print(f"\n🎯 Summary:")
print("=" * 50)
for key, value in summary_stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:,}" if isinstance(value, int) else f"{key.replace('_', ' ').title()}: {value}")


for map_file in maps_created:
    print(f"   🗺️ {map_file}")



🗺️ Generating Maps...

 Business Question 1: CUNY Routes Analysis
--------------------------------------------------
📊 Found 216,155 violations on CUNY-serving routes
Top CUNY routes by violations: {'M101': 216155}
✅ CUNY routes map generated

🚫 Business Question 2: Exempt Vehicle Violations
--------------------------------------------------
🚫 Found 425,814 exempt violations
🔍 Top repeat offenders: {'1feaad876c46cfe1cc4ee357e410ec2054c5c231d268169dfdd9527c1c084a80': 1005, '2a8ac01b157a97b3c124cf33322e71c31062b815dd3b0a9c121ee3a6c22ff64a': 611, '6d7caef8bf15e2bf8d46aa1e6dd8c0da1ae2fbc5b31acec87ab5642558d4360b': 605, 'ebabc585284d95a10b6b45730c38fa55dfd57bcc5aea304b67693b84fb4d0eb0': 588, '479d603cb0581d14fbaf8908f9f27ade6accce3a371895ba61be5cab8d23691b': 545}
✅ Exempt violations map generated!

📈 Business Question  3: Congestion Pricing Impact
--------------------------------------------------
🏙️ Found 140,248 violations in CBD
📊 Before CP (Jan 1-4): 2,018 violations
📊 After CP (Jan 5+

In [9]:
# Congestion Pricing Map 

# Date filtering 
cbd_violations = valid_coords[
    (valid_coords['Violation Latitude'].between(40.7047, 40.7614)) &
    (valid_coords['Violation Longitude'].between(-74.0200, -73.9442))
]

# Map
m = folium.Map(location=[40.7282, -73.9942], zoom_start=13)

# CBD violations
if len(cbd_violations) > 0:
    cbd_coords = [[row['Violation Latitude'], row['Violation Longitude']] 
                  for _, row in cbd_violations.iterrows()]
    
    HeatMap(cbd_coords, radius=10, blur=12, name='CBD Violations').add_to(m)

m.save('congestion_pricing_fixed.html')
print("🗺️ Congestion pricing map generated!")

🗺️ Congestion pricing map generated!
